In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nizhniy Chir,RU,48.3597,43.0865,49.14,86,100,23.11,overcast clouds
1,1,Yuzhno-Kurilsk,RU,44.0306,145.8556,47.32,99,76,10.51,broken clouds
2,2,Bonthe,SL,7.5264,-12.5050,81.09,82,22,5.88,few clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,46.51,76,0,1.14,clear sky
4,4,Bethel,US,41.3712,-73.4140,78.35,46,40,8.05,scattered clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,76.59,82,33,3.76,scattered clouds
6,6,Isangel,VU,-19.5500,169.2667,75.51,68,100,20.09,overcast clouds
7,7,Bac Lieu,VN,9.2850,105.7244,83.07,77,100,5.53,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.54,81,6,20.42,clear sky
9,9,Yellowknife,CA,62.4560,-114.3525,37.27,72,100,4.00,overcast clouds


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 60
What is your desired maximum temperature for your trip? 85


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
desired_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
desired_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bonthe,SL,7.5264,-12.5050,81.09,82,22,5.88,few clouds
4,4,Bethel,US,41.3712,-73.4140,78.35,46,40,8.05,scattered clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,76.59,82,33,3.76,scattered clouds
6,6,Isangel,VU,-19.5500,169.2667,75.51,68,100,20.09,overcast clouds
7,7,Bac Lieu,VN,9.2850,105.7244,83.07,77,100,5.53,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,77.54,81,6,20.42,clear sky
10,10,Bambous Virieux,MU,-20.3428,57.7575,75.45,88,40,1.14,scattered clouds
11,11,Butaritari,KI,3.0707,172.7902,80.67,76,99,10.00,overcast clouds
12,12,Port Elizabeth,ZA,-33.9180,25.5701,61.32,82,0,4.00,clear sky
13,13,Kitgum,UG,3.2783,32.8867,70.52,74,100,2.80,overcast clouds


In [5]:
# Determine if there are any empty rows.
desired_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = desired_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bonthe,SL,7.5264,-12.5050,81.09,82,22,5.88,few clouds
4,4,Bethel,US,41.3712,-73.4140,78.35,46,40,8.05,scattered clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,76.59,82,33,3.76,scattered clouds
6,6,Isangel,VU,-19.5500,169.2667,75.51,68,100,20.09,overcast clouds
7,7,Bac Lieu,VN,9.2850,105.7244,83.07,77,100,5.53,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
683,683,Appleton,US,44.2619,-88.4154,75.87,73,94,8.01,overcast clouds
684,684,Hue,VN,16.4667,107.6000,78.91,89,20,4.61,few clouds
685,685,Ixtapa,MX,20.7000,-105.2000,83.43,65,0,11.50,clear sky
688,688,Pisco,PE,-13.7000,-76.2167,68.02,82,5,8.05,clear sky


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bonthe,SL,81.09,few clouds,7.5264,-12.5050,
4,Bethel,US,78.35,scattered clouds,41.3712,-73.4140,
5,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,
6,Isangel,VU,75.51,overcast clouds,-19.5500,169.2667,
7,Bac Lieu,VN,83.07,overcast clouds,9.2850,105.7244,
8,Rikitea,PF,77.54,clear sky,-23.1203,-134.9692,
10,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
11,Butaritari,KI,80.67,overcast clouds,3.0707,172.7902,
12,Port Elizabeth,ZA,61.32,clear sky,-33.9180,25.5701,
13,Kitgum,UG,70.52,overcast clouds,3.2783,32.8867,


In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [9]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bonthe,SL,81.09,few clouds,7.5264,-12.5050,Bonthe Holiday Village
4,Bethel,US,78.35,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
5,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Isangel,VU,75.51,overcast clouds,-19.5500,169.2667,Tanna Lodge
7,Bac Lieu,VN,83.07,overcast clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
8,Rikitea,PF,77.54,clear sky,-23.1203,-134.9692,People ThankYou
10,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,Casa Tia Villa
11,Butaritari,KI,80.67,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
12,Port Elizabeth,ZA,61.32,clear sky,-33.9180,25.5701,39 On Nile Guest House
13,Kitgum,UG,70.52,overcast clouds,3.2783,32.8867,Jaflo Gardens Hotel


In [10]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))